In [7]:
from symfit import Variable, Parameter, symbols, Fit, Eq, Model, parameters
from kingdon import Algebra
import matplotlib.pyplot as plt
import numpy as np

In [8]:
alg = Algebra(1,1)

In [9]:
p = Parameter('p', value=2)
u = alg.vector({1: 1, 2: p})
print(u)

(1) * e1 + (p) * e2


In [10]:
usq = u*u
print(usq)
print(-usq[0])

(1 - p**2) * 1
p**2 - 1


In [11]:
s = Variable('s')

In [12]:
model_dict = {s: -usq[0]}
fit = Fit(model_dict, s=np.atleast_2d(0))

In [13]:
result = fit.execute()
print(result)


Parameter Value        Standard Deviation
p         1.000000e+00 inf
Status message         Optimization terminated successfully.
Number of iterations   5
Objective              <symfit.core.objectives.LeastSquares object at 0x00000267752933A0>
Minimizer              <symfit.core.minimizers.BFGS object at 0x0000026775292B90>

Goodness of fit qualifiers:
chi_squared            1.0136270688399157e-19
objective_value        5.068135344199579e-20
r_squared              -inf


C:\Users\martin.roelfs\Anaconda3\envs\pyga\lib\site-packages\symfit\core\fit_results.py:282: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - SS_res/SS_tot
C:\Users\martin.roelfs\Anaconda3\envs\pyga\lib\site-packages\symfit\core\fit.py:268: RuntimeWarning: divide by zero encountered in double_scalars
  s2 = rss / dof


In [14]:
# u = alg.multivector({k: Parameter(f'u{k}', min=0.1) for k in range(len(alg))})
u = alg.vector({1: Parameter('u1'), 2: Parameter('u2')})
print(u)

(u1) * e1 + (u2) * e2


In [15]:
usq = u*u
print(usq)
print(-usq[0])

(u1**2 - u2**2) * 1
-u1**2 + u2**2


In [16]:
constraints = [
#     Eq(usq[1], 0),
#     Eq(usq[2], 0),
#     Eq(usq[3], 0),
]
fit = Fit(-usq[0], constraints=constraints)

In [17]:
results = fit.execute()
print(results)


Parameter Value        Standard Deviation
u1        -3.975322e+24 nan
u2        3.975322e+24 7.071068e-01
Status message         Desired error not necessarily achieved due to precision loss.
Number of iterations   4
Objective              <symfit.core.objectives.MinimizeModel object at 0x0000026775E0C6A0>
Minimizer              <symfit.core.minimizers.BFGS object at 0x0000026775E0D000>

Goodness of fit qualifiers:
objective_value        -1.3964542131129786e+42


C:\Users\martin.roelfs\Anaconda3\envs\pyga\lib\site-packages\symfit\core\fit_results.py:119: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(self.variance(param))


In [18]:
fit.model(**results.params)

ModelOutput(variables=[_2643382523584], output=[array([-1.39645421e+42])])

In [19]:
fit.model.jacobian

[[-2*u1, 2*u2]]

In [20]:
fit.model(p=-2)

TypeError: missing a required argument: 'u1'

In [21]:
x = np.linspace(-1,1)
y, = fit.model(p=x)
plt.plot(x, y)

TypeError: missing a required argument: 'u1'

In [92]:
alg = Algebra(4)
# bvals = np.random.random(6)
B = alg.bivector({3:2, 12:2})
Bsq = B*B
print(B)
print(Bsq)
lp = (Bsq[0] + np.sqrt(Bsq[0]**2 - Bsq[15]**2))/2
lm = (Bsq[0] - np.sqrt(Bsq[0]**2 - Bsq[15]**2))/2
lp, lm
# Bsq[0]**2 - Bsq[15]**2

(2) * e12 + (2) * e34
(-8) * 1 + (8) * e1234


(-4.0, -4.0)

In [93]:
bs = parameters(' '.join(f'b_{i}' for i in range(1, 7)), value=2)

In [94]:
b = alg.bivector(bs)
bsq = b*b
bsq

MultiVector(algebra=Algebra(p=4, q=0, r=0, cse=True, precompute='none', numba=False, graded=False, simplify=True), _values=(-b_1**2 - b_2**2 - b_3**2 - b_4**2 - b_5**2 - b_6**2, 2*b_1*b_6 - 2*b_2*b_5 + 2*b_3*b_4), _keys=(0, 15))

In [95]:
bcpB = b.cp(B)
print(bcpB)
bcpB_sq = bcpB * bcpB
print(bcpB_sq)

(-2*b_3 - 2*b_4) * e13 + (2*b_2 - 2*b_5) * e14 + (2*b_2 - 2*b_5) * e23 + (2*b_3 + 2*b_4) * e24
(-2*(2*b_2 - 2*b_5)**2 + (-2*b_3 - 2*b_4)*(2*b_3 + 2*b_4) - (2*b_3 + 2*b_4)**2) * 1 + (2*(2*b_2 - 2*b_5)**2 - (-2*b_3 - 2*b_4)*(2*b_3 + 2*b_4) + (2*b_3 + 2*b_4)**2) * e1234


In [96]:
# x = alg.bivector([Variable(f"x{alg.bin2canon[i][1:]}") for i in alg.indices_for_grade[2]])
# model_dict = {x[i]: bcpB[i] for i in x.keys() if i in bcpB}
# model = Model(model_dict)
# model
# x, x1234 = symbols('x x1234', cls=Variable)
# model_dict = {x: bcpB_sq[0], x1234: bcpB_sq[15]}
# model = Model(model_dict)
# model
x, x1234 = symbols('x x1234', cls=Variable)
model_dict = {x: bsq[0], x1234: bsq[15]}
model = Model(model_dict)
model

In [97]:
constraints = [Eq(bcpB_i, 0) for bcpB_i in bcpB.values()]
fit = Fit(model_dict, **{x.name: np.atleast_2d(lp), x1234.name: np.atleast_2d(0)}, constraints=constraints)
results = fit.execute()

In [98]:
print(results)


Parameter Value        Standard Deviation
b_1       2.000000e+00 nan
b_2       2.000000e+00 nan
b_3       2.000000e+00 nan
b_4       2.000000e+00 nan
b_5       2.000000e+00 nan
b_6       2.000000e+00 nan
Status message         Singular matrix C in LSQ subproblem
Number of iterations   1
Objective              <symfit.core.objectives.LeastSquares object at 0x0000026778F5F6A0>
Minimizer              <symfit.core.minimizers.SLSQP object at 0x0000026778F5F730>

Goodness of fit qualifiers:
chi_squared            464.0
objective_value        232.0
r_squared              -inf

Constraints:
--------------------
Question: -2*b_3 - 2*b_4 == 0?
Answer:   -8.0

Question: 2*b_2 - 2*b_5 == 0?
Answer:   0.0

Question: 2*b_2 - 2*b_5 == 0?
Answer:   0.0

Question: 2*b_3 + 2*b_4 == 0?
Answer:   8.0




In [88]:
model(**results.params)

ModelOutput(variables=[x, x1234], output=[array([-2.00000014]), array([2.00000014])])

In [89]:
b_re = b(**results.params)
print(b_re)
print()
print(b_re*b_re)
print()
print(b_re.cp(B))

(1.0000000473846042) * e12 + (-4.219012822309351e-22) * e13 + (5.082197683525802e-21) * e14 + (-6.776263578034403e-21) * e23 + (-8.438025644618822e-22) * e24 + (1.0000000241544578) * e34

(-2.000000143078127) * 1 + (0.0) * e12 + (0.0) * e13 + (0.0) * e14 + (0.0) * e23 + (-1.504632769052528e-36) * e24 + (0.0) * e34 + (2.0000001430781262) * e1234

(8.470329472543003e-21) * e13 + (1.2657038466928294e-21) * e14 + (1.2037062152420224e-35) * e23 + (3.3881317890172014e-21) * e24
